In [1]:
import numpy as np
import scipy as sp

In [2]:
import pymolviz as pmv

In [3]:
origin = np.array([-20.0, -20.0, -20.0])
step_sizes = np.array([1, 1, 1])
step_counts = np.array([40, 40, 40])

x  = np.linspace(origin[0], origin[0] + step_sizes[0] * step_counts[0], step_counts[0])
y  = np.linspace(origin[1], origin[1] + step_sizes[1] * step_counts[1], step_counts[1])
z  = np.linspace(origin[2], origin[2] + step_sizes[2] * step_counts[2], step_counts[2])
xx, yy, zz = np.meshgrid(x, y, z)
positions = np.array([xx.flatten(), yy.flatten(), zz.flatten()]).T

In [4]:
def to_spherical(positions):
    r = np.sqrt(np.sum(positions**2, axis=1))
    theta = np.arccos(positions[:, 2] / r)
    phi = np.arctan2(positions[:, 1], positions[:, 0])
    return np.array([r, theta, phi]).T

In [5]:
spherical = to_spherical(positions)

In [6]:
spherical

array([[34.64101615,  2.18627604, -2.35619449],
       [34.0591588 ,  2.16168597, -2.35619449],
       [33.49860409,  2.13625713, -2.35619449],
       ...,
       [33.49860409,  1.00533552,  0.78539816],
       [34.0591588 ,  0.97990668,  0.78539816],
       [34.64101615,  0.95531662,  0.78539816]])

In [7]:
def N(m, l):
    m = np.abs(m)
    return np.sqrt(2 * l + 1) * np.sqrt(np.math.factorial(l - m) / np.math.factorial(l + m))



In [8]:
l = 2

In [9]:
legendre_results = np.array([sp.special.lpmn(l, l, np.cos(theta))[0] for theta in spherical[:, 1]])

In [10]:
legendre_results
def Y(m, l, theta, phi):
    return N(m, l) * legendre_results[:, m, l] * np.exp(1j * m * phi)
values = {}

for m in range(-l, l + 1):
    values[(m, l)] = Y(np.abs(m), l, spherical[:, 1], spherical[:, 2])

In [11]:
reg_datas = {key : pmv.RegularData(np.abs(val)**2, origin=origin, step_sizes=step_sizes, step_counts=step_counts) for key, val in values.items()}

In [12]:
reg_datas

{(-2, 2): <pymolviz.volumetric.RegularData.RegularData at 0x7f561050a940>,
 (-1, 2): <pymolviz.volumetric.RegularData.RegularData at 0x7f55a99b98b0>,
 (0, 2): <pymolviz.volumetric.RegularData.RegularData at 0x7f55a99b9bb0>,
 (1, 2): <pymolviz.volumetric.RegularData.RegularData at 0x7f55a99b99d0>,
 (2, 2): <pymolviz.volumetric.RegularData.RegularData at 0x7f55a99b9e50>}

In [13]:
iso_vols = {key : pmv.IsoVolume(reg_data, f"{key[0]}_{key[1]}", clims = np.linspace(0,3 ,100)) for key, reg_data in reg_datas.items()}

In [14]:
pmv.Script(iso_vols.values()).write("orbitals.py")

In [15]:
distance_values = np.linalg.norm(positions, axis=1)

In [16]:
reg2_data = pmv.RegularData(distance_values, origin=origin, step_sizes=step_sizes, step_counts=step_counts)

In [17]:

iso_surfs = {key : pmv.IsoSurface(reg2_data, 5, f"{key[0]}_{key[1]}_surface", color = pmv.ColorRamp(reg_data)) for key, reg_data in reg_datas.items()}

TypeError: 'module' object is not callable

In [ ]:
g = pmv.Group(iso_surfs.values(), "orbitals")
g.write("orbital_surfs.py")